# Imports

In [ ]:
import sys
sys.path.append('..')
from _utils import flatten_list
from training_helpers import load_data_pairs

In [ ]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import os

# Load data

In [ ]:
data_dict = load_data_pairs()
pos_pairs = flatten_list([data_dict[x] for x in data_dict if "pos" in x])
neg_pairs = flatten_list([data_dict[x] for x in data_dict if "neg" in x])

In [14]:
testads = pd.DataFrame(load_json("../00_data/EURES/eures_testads_final_short.json"))

# Pre-Training

In [ ]:
pretraining_list = list(set(flatten_list(pos_pairs)))
len(pretraining_list)

In [ ]:
# jobgbert model
#model = SentenceTransformer("agne/jobgbert")
#gbert model 
model = SentenceTransformer("deepset/gbert-base")

##  TSDAE

In [6]:
data_dict = load_data_pairs()
pos_pairs = flatten_list([data_dict[x] for x in data_dict if "pos" in x])
neg_pairs = flatten_list([data_dict[x] for x in data_dict if "neg" in x])

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

pretraining_dataset = datasets.DenoisingAutoEncoderDataset(pretraining_list)
train_dataloader = DataLoader(pretraining_dataset, batch_size=16, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs = 5,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)
model.save("../00_data/SBERT_Models/models/gbert_TSDAE_epochs5")

## Pretrained Model

In [ ]:
def create_trainig_samples(pos_dev_samples,neg_pairs):
  dev_set_total =[]
  anchors = set([x[0] for x in pos_dev_samples])
  neg_dev_samples = [x for x in neg_pairs if x[0] in anchors]
  print("Creating Dev Set")
  for anchor in tqdm(anchors):
    pos_pairs_filtered = [x[1] for x in pos_dev_samples if x[0]==anchor]
    neg_pairs_filtered = [x[1] for x in neg_dev_samples if x[0]==anchor]
    dev_set_total.append({"query":anchor,"positive":pos_pairs_filtered,"negative":neg_pairs_filtered})
  return dev_set_total

# Save the Model

In [2]:
# %%capture
# !pip install sentence_transformers